In [1]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
import seaborn as sbn
import pickle as pkl 
import datetime as dt
import random

pd.set_option('display.max_columns', 100)

# reproducibility
random.seed(0)
np.random.seed(0)

In [2]:
info = pd.read_csv('../data/raw/ccle_info.txt')
info.head()

,DepMap_ID,cell_line_name,stripped_cell_line_name,CCLE_Name,Alias,COSMICID,sex,source,Achilles_n_replicates,cell_line_NNMD,culture_type,culture_medium,cas9_activity,RRID,WTSI_Master_Cell_ID,sample_collection_site,primary_or_metastasis,primary_disease,Subtype,age,Sanger_Model_ID,depmap_public_comments,lineage,lineage_subtype,lineage_sub_subtype,lineage_molecular_subtype
0,ACH-000001,NIH:OVCAR-3,NIHOVCAR3,NIHOVCAR3_OVARY,OVCAR3,905933.0,Female,ATCC,NaN,NaN,NaN,NaN,NaN,CVCL_0465,2201.0,ascites,Metastasis,Ovarian Cancer,"Adenocarcinoma, high grade serous",60.0,SIDM00105,NaN,ovary,ovary_adenocarcinoma,high_grade_serous,NaN
1,ACH-000002,HL-60,HL60,HL60_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,905938.0,Female,ATCC,NaN,NaN,NaN,NaN,NaN,CVCL_0002,55.0,haematopoietic_and_lymphoid_tissue,Primary,Leukemia,"Acute Myelogenous Leukemia (AML), M3 (Promyelo...",35.0,SIDM00829,NaN,blood,AML,M3,NaN
2,ACH-000003,CACO2,CACO2,CACO2_LARGE_INTESTINE,"CACO2, CaCo-2",NaN,Male,ATCC,NaN,NaN,NaN,NaN,NaN,CVCL_0025,NaN,Colon,NaN,Colon/Colorectal Cancer,Adenocarcinoma,NaN,SIDM00891,NaN,colorectal,colorectal_adenocarcinoma,NaN,NaN
3,ACH-000004,HEL,HEL,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,907053.0,Male,DSMZ,2.0,-3.079202,Suspension,RPMI + 10% FBS,52.4,CVCL_0001,783.0,haematopoietic_and_lymphoid_tissue,NaN,Leukemia,"Acute Myelogenous Leukemia (AML), M6 (Erythrol...",30.0,SIDM00594,NaN,blood,AML,M6,NaN
4,ACH-000005,HEL 92.1.7,HEL9217,HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,NaN,Male,ATCC,2.0,-2.404409,Suspension,RPMI + 10% FBS,86.6,CVCL_2481,NaN,bone_marrow,NaN,Leukemia,"Acute Myelogenous Leukemia (AML), M6 (Erythrol...",30.0,SIDM00593,NaN,blood,AML,M6,NaN


In [3]:
lincs_cells = pd.read_csv('../../LINCS/LVL3/data/raw/cellinfo_beta.txt', sep='\t')
lincs_cells.head()

,cell_iname,cellosaurus_id,donor_age,donor_age_death,donor_disease_age_onset,doubling_time,growth_medium,provider_catalog_id,feature_id,cell_type,donor_ethnicity,donor_sex,donor_tumor_phase,cell_lineage,primary_disease,subtype,provider_name,growth_pattern,ccle_name,cell_alias
0,1HAE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,normal,Unknown,Unknown,Unknown,unknown,unknown,normal fibroblast sample,NaN,unknown,NaN,NaN
1,AALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,normal,Unknown,Unknown,Unknown,unknown,unknown,normal epithelium sample,NaN,unknown,NaN,NaN
2,AG06263_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tumor,Unknown,Unknown,Unknown,unknown,unknown,unknown,NaN,unknown,NaN,NaN
3,AG06840_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tumor,Unknown,Unknown,Unknown,unknown,unknown,unknown,NaN,unknown,NaN,NaN
4,AG078N1_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tumor,Unknown,Unknown,Unknown,unknown,unknown,unknown,NaN,unknown,NaN,NaN


In [4]:
with open('../../PPI_graph.pkl', 'rb') as f: 
    PPI_graph = pkl.load(f)

In [5]:
ccle_expr = pd.read_csv('../data/raw/ccle_expression.txt')#.rename({'Unnamed:0':'DepMap_ID'}, axis=1)
ccle_expr.columns = ['DepMap_ID'] + [x.split(' ')[0].upper() for x in ccle_expr.columns[1:]]
ccle_genes = list(ccle_expr.columns[1:])
ccle_expr.head()

,DepMap_ID,TSPAN6,TNMD,DPM1,SCYL3,C1ORF112,FGR,CFH,FUCA2,GCLC,NFYA,STPG1,NIPAL3,LAS1L,ENPP4,SEMA3F,CFTR,ANKIB1,CYP51A1,KRIT1,RAD52,BAD,LAP3,CD99,HS3ST1,AOC1,WNT16,HECW1,MAD1L1,LASP1,SNX11,TMEM176A,M6PR,KLHL13,CYP26B1,ICA1,DBNDD1,ALS2,CASP10,CFLAR,TFPI,NDUFAF7,RBM5,MTMR7,SLC7A2,ARF5,SARM1,POLDIP2,PLXND1,AK2,...,FAM240A,FAM95C,LITAFD,PRRT1B,BX276092.9,ETDC,LMLN2,MYOCOS,HSFX3,VSIG10L2,PRSS50,CPHXL,AC131160.1,TPTEP2-CSNK1E,GNG14,SLURP2,AC069544.2,SCO2,C2ORF81,PERCC1,THSD8,LYNX1-SLURP2,OR8B3,OR4F16,OR8B2,TMEM247,SMIM38,OR8S1,OR4F29,EEF1AKMT4,AC022414.1,TBCE,SMIM41,AC008397.1,GCSAML-AS1,CCDC39,EEF1AKMT4-ECE2,AP000812.4,UPK3BL2,AC093512.2,ARHGAP11B,AC004593.2,AC090517.4,AL160269.1,ABCF2-H2BE1,POLR2J3,H2BE1,AL445238.1,GET1-SH3BGR,AC113348.1
0,ACH-001113,4.990501,0.000000,7.273702,2.765535,4.480265,0.028569,1.269033,3.058316,6.483171,5.053980,3.456806,4.415488,4.766595,2.280956,3.237258,0.000000,5.125982,6.636770,5.638364,3.881665,5.156639,4.775051,5.904966,0.097611,0.111031,0.042644,2.847997,3.336283,5.371210,4.313971,0.000000,7.536830,5.207893,2.965323,1.922198,2.049631,4.478972,2.077243,5.101818,0.056584,4.614710,5.286142,0.545968,1.613532,7.381197,2.611172,5.929081,5.293885,6.730640,...,0.0,2.724650,0.042644,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.748461,0.042644,1.782409,0.356144,0.650765,0.411426,2.895303,0.765535,1.731183,0.056584,0.695994,0.070389,0.0,0.070389,0.0,0.0,0.389567,0.0,0.070389,4.578335,0.000000,5.761019,0.0,0.000000,0.0,0.028569,0.000000,1.464668,5.234961,4.139961,1.214125,0.000000,0.111031,0.150560,1.427606,5.781884,0.0,0.000000,0.799087,0.000000
1,ACH-001289,5.209843,0.545968,7.070604,2.538538,3.510962,0.000000,0.176323,3.836934,4.200850,3.832890,1.910733,3.374344,4.861955,3.625270,1.275007,0.028569,5.177121,7.130313,5.061776,3.023255,5.542258,6.305423,6.641546,0.084064,0.014355,0.275007,0.189034,2.903038,4.955127,4.421560,0.042644,7.133091,2.861955,0.124328,3.513491,4.056584,4.286142,0.333424,4.520422,0.070389,3.987321,6.192391,3.163499,2.185867,7.792530,2.427606,6.269407,3.785551,7.327059,...,0.0,0.918386,0.000000,0.042644,0.000000,0.0,0.0,0.000000,0.111031,0.0,0.000000,0.000000,1.799087,0.526069,0.000000,0.000000,4.202418,0.298658,0.765535,0.000000,4.173927,0.189034,0.0,0.014355,0.0,0.0,0.028569,0.0,0.014355,2.182692,0.042644,5.771357,0.0,0.000000,0.0,1.090853,0.000000,1.490570,0.941106,4.107688,1.835924,0.000000,0.310340,0.000000,0.807355,4.704319,0.0,0.000000,0.464668,0.070389
2,ACH-001339,3.779260,0.000000,7.346425,2.339137,4.254745,0.056584,1.339137,6.724241,3.671293,3.775051,2.895303,3.613532,4.300856,0.799087,0.275007,0.042644,4.149747,5.655352,4.858976,2.675816,4.560715,6.170125,8.182245,0.389567,0.084064,0.084064,0.084064,5.733625,6.274262,4.407353,0.214125,7.361417,0.137504,0.454176,2.301588,3.317594,3.746313,1.232661,4.750607,3.914565,3.723559,5.008541,0.545968,1.400538,6.864558,2.241840,6.123087,7.242793,8.119875,...,0.0,0.000000,0.000000,0.000000,0.028569,0.0,0.0,0.028569,0.028569,0.0,0.014355,0.000000,0.992768,0.201634,0.000000,0.014355,3.209453,0.344828,0.555816,0.000000,2.952334,0.070389,0.0,0.014355,0.0,0.0,0.028569,0.0,0.014355,3.012569,0.000000,4.744699,0.0,0.000000,0.0,0.000000,0.000000,0.985500,1.124328,2.313246,1.823749,0.084064,0.176323,0.042644,1.384050,4.931683,0.0,0.028569,0.263034,0.000000
3,ACH-001538,5.726831,0.000000,7.086189,2.543496,3.102658,0.000000,5.914565,6.099716,4.475733,4.294253,2.472488,4.573496,5.314697,3.488001,2.980025,0.028569,3.872829,6.176921,3.714795,2.726831,5.565293,5.230357,6.811728,2.657640,0.084064,0.124328,0.367371,4.161888,6.703211,3.541019,0.070389,7.208478,0.014355,1.124328,3.946731,3.237258,2.931683,2.797013,4.591560,3.122673,3.554589,5.383359,0.111031,1.084064,7.403779,1.974529,5.444270,0.970854,7.954487,...,0.0,0.641546,0.000000,0.000000,0.014355,0.0,0.0,0.028569,0.000000,0.0,0.000000,0.000000,0.641546,0.042644,0.000000,1.454176,2.438293,0.678072,1.117695,0.000000,0.000000,0.150560,0.0,0.000000,0.0,0.0,0.124328,0.0,0.000000,3.000000,0.000000,4

In [6]:
# Step 1: what cell lines are in LINCS? 
linc_lines = lincs_cells[['cell_iname', 'ccle_name']].merge(info[['CCLE_Name', 'DepMap_ID']], left_on='ccle_name', right_on='CCLE_Name')
linc_lines.head()

,cell_iname,ccle_name,CCLE_Name,DepMap_ID
0,OCILY10,OCILY10_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,OCILY10_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,ACH-001146
1,TC32,TC32_BONE,TC32_BONE,ACH-001205
2,HCC515,HCC515_LUNG,HCC515_LUNG,ACH-000872
3,SALE,SALE_LUNG,SALE_LUNG,ACH-000064
4,SHSY5Y,SHSY5Y_AUTONOMIC_GANGLIA,SHSY5Y_AUTONOMIC_GANGLIA,ACH-001188


In [7]:
overlap_genes = set(PPI_graph['nodelist']).intersection(set(ccle_genes))
len(overlap_genes)

13283

In [8]:
len(PPI_graph['nodelist'])

13565

In [9]:
ccle_expr_zscored = ccle_expr

# zscore 
ccle_expr_zscored[ccle_genes] = (ccle_expr_zscored[ccle_genes].values - ccle_expr_zscored[ccle_genes].values.mean(axis=0))/ccle_expr_zscored[ccle_genes].values.std(axis=0)

# merge with lincs cell names 
ccle_expr_zscored = ccle_expr_zscored.merge(linc_lines, on='DepMap_ID')

# select PPI graph genes 
ccle_expr_zscored = ccle_expr_zscored[['DepMap_ID', 'cell_iname', 'ccle_name'] + list(overlap_genes)]

print('shape', ccle_expr_zscored.shape)
ccle_expr_zscored.head()

shape (133, 13286)


,DepMap_ID,cell_iname,ccle_name,ST3GAL2,GRIN1,SLIT2,EGLN2,PEX13,KCNH8,COL4A5,LMBRD1,RUVBL1,SRXN1,TFEC,LTO1,CFP,RPRD2,ZNF169,ADGRB3,CBL,EHBP1,MAST1,MTHFR,TMSB10,CHST5,KXD1,DGKH,EGFL8,SELENON,SP140,GJA5,UAP1L1,DNAI1,ZNF814,ZBTB17,SCTR,ZNF354C,AKIP1,EGR3,KIF4B,TMIGD3,MOB4,PPP1R14A,LYRM4,GNGT2,PPP2R5C,BBS5,PTPN7,FBXO6,GCLC,...,CTSS,ALAS2,MAF,OR8J1,IGSF8,PACSIN3,IFNA4,FFAR3,HYOU1,AP1B1,SGIP1,CBX5,MYH15,ARHGAP26,RPS21,OR5M9,PLCG1,WDR41,ARID5B,ASPM,SLC4A2,CRYBA2,NAP1L4,ZNF565,TNFRSF11B,TRIP6,BAG3,CD24,RPL24,WDTC1,GHITM,PINX1,PLAAT4,HOXA2,TRHDE,ATP6V1B1,PNMA1,PCCB,PLXNA1,GSK3B,SNRPD1,INTS2,CXCR4,KLC3,ZNF365,ZNF740,ECM1,MYOZ2,ATP6V0C,CSF2RB
0,ACH-000123,COV434,COV434_OVARY,0.308293,0.029133,-0.149332,-0.622608,2.109737,-0.244858,1.148072,-0.569996,0.022964,-0.316437,-0.314477,-0.873786,-0.261498,0.010050,0.094754,1.435845,0.796264,-0.656091,0.986760,0.778326,-1.486876,-0.487234,0.063411,-0.219447,-0.323744,0.742922,-0.568103,-0.342143,-0.247021,-0.508756,0.528948,-0.326922,-0.339212,1.218003,1.452817,-0.510043,-0.027840,0.568765,0.119529,0.603517,-0.117628,-0.469200,0.166790,0.896044,-0.414308,1.168229,-1.379842,...,-1.080364,0.015214,-0.634084,-0.053919,-0.786133,0.787870,-0.104702,-0.241910,-0.413598,0.389321,-0.533022,0.689149,-0.608955,-1.389345,-0.690005,-0.072929,-0.558275,-0.147552,3.202295,-0.530673,0.606923,-0.448273,0.452219,-0.316291,-0.132816,-2.099342,-0.230162,-1.109056,-0.122863,-0.350468,-0.261988,0.375490,-0.619965,-0.893428,-0.519380,0.211129,0.400512,1.578057,-0.682470,-0.562465,-0.300559,-0.148964,-0.305835,-0.440886,-0.455093,-0.229296,-1.171512,-0.161561,0.402694,-0.365711
1,ACH-000465,SKMEL1,SKMEL1_SKIN,1.002036,-0.683018,-0.035610,1.289642,-0.955581,0.544522,-1.170356,-0.737431,-0.744249,-0.474254,-0.222868,-0.059288,-0.449767,-0.896285,-1.069726,-0.164731,0.742613,0.996012,0.686164,1.096561,-1.827324,0.060776,0.884327,-0.045098,-0.221243,1.235396,-0.848681,-0.380149,2.293523,-0.171288,0.735416,0.938012,-0.299863,0.745654,0.708787,3.625428,0.776908,-0.131072,0.477550,-0.592164,0.360353,-0.526862,-2.972592,1.646099,-0.165629,1.116914,-0.074435,...,-0.854341,-0.190008,-0.128647,-0.053919,2.353328,1.244539,-0.104702,0.076027,0.007392,1.897811,0.042712,0.414003,-0.695517,-0.073870,0.077024,-0.072929,2.018530,-0.322697,0.965510,-0.754514,1.268144,-0.448273,2.328345,-0.381683,0.243400,-2.014144,0.634769,-1.315049,-0.943167,1.250493,0.510878,2.345159,-1.168007,-0.726712,-0.546379,-0.644031,-1.227850,0.341964,2.264866,0.837974,-0.878977,0.554996,-0.775009,-0.226745,2.380571,-0.873418,1.424484,-0.185402,1.780890,-0.365711
2,ACH-000999,SNU1040,SNU1040_LARGE_INTESTINE,-0.807224,0.145635,-0.414071,0.274888,0.296583,0.399910,-0.868891,0.708434,1.126769,0.952953,-0.397951,0.080261,-0.645772,0.458910,-0.111461,-0.409548,0.367298,0.180324,-0.051144,0.188384,-0.188274,0.584490,0.563205,0.012051,-0.120067,0.394558,-0.671676,0.193627,0.508319,-0.374128,-0.502798,0.069870,-0.108784,-0.876789,0.030122,-0.445147,-0.147044,-0.196520,-0.013882,0.864936,-0.127888,-0.450703,1.038819,0.928466,-0.391457,1.992813,0.432799,...,-1.133223,-0.190008,0.045222,-0.053919,1.315036,1.016542,-0.104702,-0.241910,1.181839,0.214126,-0.484775,-0.340810,-0.643080,0.259435,0.498065,-0.072929,0.348954,-0.194283,-1.248385,-1.239608,0.109354,1.393199,0.306835,0.187338,-0.194645,0.819895,0.510243,0.443006,-0.037145,0.958024,0.996063,-0.172958,-1.062957,-0.958420,-0.573908,0.321418,-0.382475,2.378465,-0.154600,1.570402,0.597254,0.034146,0.291371,1.213802,2.996629,1.040858,-0.486868,-0.283115,0.659535,-0.365711
3,ACH-000017,SKBR3,SKBR3_BREAST,-2.016029,1.361690,-0.823237,0.165522,-0.103936,-0.435080,-0.314680,-0.544161,-1.045753,0.921816,-0.369582,-0.396651,-0.522471,0.092567,1.115010,-0.426114,-0.532899,-0.399915,-0.470171,-0.388019,-0.851854,-0.354696,-0.822275,-0.187659,-1.159396,-0.186145,-0.601525,4.784246,1.613273,2.079951,-0.438200,-0.394547,-0.299863,-0.819035,-0.156465,-0.606034,0.090265,-0.131072,-0.083675,-0.531011,-2.295716,-0.507266,-1.064128,-0.527

In [10]:
ccle_expr_zscored.isna().any().any()

False

In [15]:
expr  =    {'unpert_expr'      : ccle_expr_zscored, 
            'linc_ccle_lines'  : linc_lines,        
            'meta'             : 'z-scored CCLE rna expression', 
            'source'           : './CCLE/notebooks/make_unpert_expression_data.ipynb',
            'creation_date'    : dt.datetime.now().__str__()} 

with open('../../cell_baseline.pkl', 'wb') as f: 
    pkl.dump(expr, f)